##  FastAPI DEPENDS 

Dependency  means we can inject functions on some class

for examples 
 engine is a part of car 
 in traditional system -> 
```   class car 
           engine defines under class car 

 class Engine:
    def start(self):
        return "Vroom!"

class Car:
    def __init__(self):
        # The Car is "Hard-Coupled" to the Engine
        self.engine = Engine() 

    def drive(self):
        print(self.engine.start()) ``` 


problem : if we want to change the engine to Electrict enginer or turboengine , we have to modify the car class code itself ... its voilates Open closed principle (coded should be open for extension but closed for modification)




## in dependency injection

In DI, the Car doesn't care how the engine is made; it only cares that it receives something that acts like an engine.

In [4]:
class Engine:
    def start(self):
        return "Standard Engine started"

class ElectricEngine:
    def start(self):
        return "Silent Electric Motor started"

class Car:
    def __init__(self, engine): # Injecting the dependency
        self.engine = engine

    def drive(self):
        print(self.engine.start())

# --- Usage ---
std_engine = Engine()
tesla_engine = ElectricEngine()

# We "Inject" the dependency at runtime
my_car = Car(std_engine)
my_electric_car = Car(tesla_engine)

***********************************************************************

class Engine:
    def start(self):
        return "Vroom!"

class Car:
    def __init__(self):
        # The Car is "Hard-Coupled" to the Engine
        self.engine = Engine() 

    def drive(self):
        print(self.engine.start())

## data base configs (sql lite)


In [2]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine
from sqlalchemy.orm import sessionmaker, declarative_base, relationship

# ---------------- DB CONFIG ----------------
DATABASE_URL = "sqlite:///./app.db"

engine = create_engine(
    DATABASE_URL,
    connect_args={"check_same_thread": False}
)

SessionLocal = sessionmaker(bind=engine)
Base = declarative_base()

# ---------------- MODELS ----------------
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True)
    email = Column(String, unique=True, index=True)

    messages = relationship("Message", back_populates="user")


class Message(Base):
    __tablename__ = "messages"

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey("users.id")) 

    user = relationship("User", back_populates="messages")


# ---------------- CREATE TABLES ----------------
Base.metadata.create_all(engine)

# ---------------- DEPENDENCY ----------------
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


part2 now add fastapi

### problem was ... main_database_fastapi.py files 
was creating app.db files under (fastapiforml folder 
but But this notebook was  creating a database file under basic jan api.py

but i want both code acesss same database so below code 

In [17]:
from pathlib import Path

ROOT_DIR = Path.cwd().parent

print("CWD:", Path.cwd())
print("ROOT_DIR:", ROOT_DIR)
print("DB PATH:", ROOT_DIR / "app.db")
print("DB EXISTS:", (ROOT_DIR / "app.db").exists())


CWD: /home/ujjwal/learning/fastapi_for_ml_ai_enginners /basic_genai_fastapi
ROOT_DIR: /home/ujjwal/learning/fastapi_for_ml_ai_enginners 
DB PATH: /home/ujjwal/learning/fastapi_for_ml_ai_enginners /app.db
DB EXISTS: False


In [ ]:
import asyncio
from pathlib import Path
import uvicorn
from fastapi import FastAPI, Depends, status, HTTPException
from fastapi.responses import RedirectResponse
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine
from sqlalchemy.orm import sessionmaker, declarative_base, relationship, Session

# --- 1. CLEANUP PREVIOUS RUNS ---

# --- DATABASE SETUP ---
from pathlib import Path

# notebook.ipynb → basic → ROOT
ROOT_DIR = Path.cwd().parent

DATABASE_URL = f"sqlite:///{ROOT_DIR}/app.db"



engine = create_engine(
    DATABASE_URL, connect_args={"check_same_thread": False}
)
SessionLocal = sessionmaker(bind=engine, autocommit=False, autoflush=False)
Base = declarative_base()


def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


# --- MODELS ---
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    email = Column(String, unique=True, index=True, nullable=False)

    messages = relationship("Message", back_populates="user")


class Message(Base):
    __tablename__ = "messages"

    id = Column(Integer, primary_key=True, index=True)
    content = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("users.id"))

    user = relationship("User", back_populates="messages")


Base.metadata.create_all(bind=engine)

# --- APP ---
app = FastAPI()


@app.get("/", include_in_schema=False)
def docs_redirect_controller():
    return RedirectResponse(url="/docs", status_code=status.HTTP_303_SEE_OTHER)


@app.post("/users")
def create_user(email: str, db: Session = Depends(get_db)):
    if db.query(User).filter(User.email == email).first():
        raise HTTPException(status_code=400, detail="User already exists")

    user = User(email=email)
    db.add(user)
    db.commit()
    db.refresh(user)
    return user


@app.post("/users/{email}/messages")
def create_message(email: str, content: str, db: Session = Depends(get_db)):
    user = db.query(User).filter(User.email == email).first()
    if not user:
        raise HTTPException(status_code=404, detail="User not found")

    msg = Message(content=content, user_id=user.id)
    db.add(msg)
    db.commit()
    db.refresh(msg)
    return msg


@app.get("/users/{email}/messages")
def get_messages(email: str, db: Session = Depends(get_db)):
    user = db.query(User).filter(User.email == email).first()
    if not user:
        raise HTTPException(status_code=404, detail="User not found")

    return user.messages


# --- SERVER ---
if 'server' in globals():
    server.should_exit = True
    # Give it a moment to release the port
    await asyncio.sleep(1) 


config = uvicorn.Config(app, host="127.0.0.1", port=8000)
server = uvicorn.Server(config)

loop = asyncio.get_event_loop()
loop.create_task(server.serve())

print("Server running on http://127.0.0.1:8000")


INFO:     Started server process [101419]
INFO:     Waiting for application startup.


Server running on http://127.0.0.1:8000


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51812 - "GET / HTTP/1.1" 303 See Other
INFO:     127.0.0.1:51812 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51812 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:52184 - "POST /users?email=ujjwal%40gmail.com HTTP/1.1" 200 OK
INFO:     127.0.0.1:46656 - "POST /users/ujjwal%40gmail.com/messages?content=fadfad HTTP/1.1" 200 OK
INFO:     127.0.0.1:51786 - "POST /users/ujjwal%40gmail.com/messages?content=fadfadfasdfadads HTTP/1.1" 200 OK
INFO:     127.0.0.1:51786 - "POST /users/ujjwal%40gmail.com/messages?content=fadsfafada HTTP/1.1" 200 OK
INFO:     127.0.0.1:51786 - "POST /users/ujjwal%40gmail.com/messages?content=fadsfafada HTTP/1.1" 200 OK
INFO:     127.0.0.1:51786 - "POST /users/ujjwal%40gmail.com/messages?content=fadsfafada HTTP/1.1" 200 OK
INFO:     127.0.0.1:51786 - "POST /users/ujjwal%40gmail.com/messages?content=fadsfafada HTTP/1.1" 200 OK
INFO:     127.0.0.1:59982 - "GET /users/ujjwal%40gmail.com/messages HTTP/1.1" 200 OK
